In [2]:
import pandas as pd
import plotly.express as px

# Load Excel
xls = pd.ExcelFile("Enrolment Data vs Attendance Report.xlsx")
enrol_df = xls.parse("Enrolment Data")
attend_df = xls.parse("Attendance Report")

# Ensure numeric columns
for df in [enrol_df, attend_df]:
    for col in ["Boys", "Girls", "Total"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Merge datasets
common_cols = ["School_Name", "Grade_Level", "Education_Level", "Term", "Year"]
merged_df = pd.merge(attend_df, enrol_df, on=common_cols, suffixes=('_Attendance', '_Enrolment'))

# Calculate grade-level attendance rate
merged_df["Attendance Rate (%)"] = (merged_df["Total_Attendance"] / merged_df["Total_Enrolment"]) * 100
merged_df["Attendance Rate (%)"] = merged_df["Attendance Rate (%)"].fillna(0)
merged_df["Rate_Label"] = merged_df["Attendance Rate (%)"].round(0).astype(int).astype(str) + "%"

# Define expected schools
school_map = {
    "ECDE": [
        "Kalobeyei Morning Star Sch", "Kalobeyei Settlement Sch", "Kalobeyei Friends Sch",
        "Joy Sch", "Future Sch", "Bright Sch", "Nationokar Sch", "Esikiriat Sch"
    ],
    "Primary": [
        "Kalobeyei Morning Star Sch", "Kalobeyei Settlement Sch", "Kalobeyei Friends Sch",
        "Joy Sch", "Future Sch", "Bright Sch", "Nationokar Sch", "Esikiriat Sch"
    ],
    "Junior": [
        "Kalobeyei Morning Star Sch", "Kalobeyei Settlement Sch", "Kalobeyei Friends Sch",
        "Joy Sch", "Future Sch", "Bright Sch", "Nationokar Sch", "Esikiriat Sch"
    ],
    "Secondary": [
        "Kalobeyei Settlement Secondary",
        "Brightstar Integrated Secondary",
        "The Big Heart Foundation Girls"
    ]
}

# Process per education level
for level, expected_schools in school_map.items():
    df_subset = merged_df[merged_df["Education_Level"] == level].copy()

    # Fill missing schools
    all_schools_df = pd.DataFrame(expected_schools, columns=["School_Name"])
    df_subset = all_schools_df.merge(df_subset, on="School_Name", how="left")

    # Fill blanks
    df_subset["Attendance Rate (%)"] = df_subset["Attendance Rate (%)"].fillna(0)
    df_subset["Rate_Label"] = df_subset["Attendance Rate (%)"].round(0).astype(int).astype(str) + "%"
    df_subset["Total_Attendance"] = df_subset["Total_Attendance"].fillna(0)
    df_subset["Total_Enrolment"] = df_subset["Total_Enrolment"].fillna(0)
    df_subset["Grade_Level"] = df_subset["Grade_Level"].fillna("N/A")
    df_subset["Education_Level"] = level

    # ------------------------
    # 📊 Chart 1: By Grade
    # ------------------------
    fig1 = px.bar(
        df_subset,
        x="School_Name",
        y="Attendance Rate (%)",
        color="Grade_Level",
        barmode="group",
        text="Rate_Label",
        hover_data={
            "School_Name": True,
            "Education_Level": True,
            "Grade_Level": True,
            "Total_Enrolment": True,
            "Total_Attendance": True
        },
        title=f"Attendance Rate per Grade and School — {level}"
    )
    fig1.update_layout(
        xaxis_title="School Name",
        yaxis_title="Attendance Rate (%)",
        xaxis_tickangle=-45,
        height=550
    )
    fig1.show()

    # ------------------------
    # 📊 Chart 2: Boys, Girls & Average Attendance
    # ------------------------

    # Fill gender values
    df_subset["Boys_Attendance"] = df_subset["Boys_Attendance"].fillna(0)
    df_subset["Girls_Attendance"] = df_subset["Girls_Attendance"].fillna(0)
    df_subset["Boys_Enrolment"] = df_subset["Boys_Enrolment"].fillna(0)
    df_subset["Girls_Enrolment"] = df_subset["Girls_Enrolment"].fillna(0)

    # Boys
    boys = df_subset.groupby("School_Name")[["Boys_Attendance", "Boys_Enrolment"]].sum().reset_index()
    boys["Gender"] = "Boys"
    boys["Rate"] = (boys["Boys_Attendance"] / boys["Boys_Enrolment"].replace(0, pd.NA)) * 100
    boys = boys.rename(columns={"Boys_Attendance": "Attendance", "Boys_Enrolment": "Enrolment"})

    # Girls
    girls = df_subset.groupby("School_Name")[["Girls_Attendance", "Girls_Enrolment"]].sum().reset_index()
    girls["Gender"] = "Girls"
    girls["Rate"] = (girls["Girls_Attendance"] / girls["Girls_Enrolment"].replace(0, pd.NA)) * 100
    girls = girls.rename(columns={"Girls_Attendance": "Attendance", "Girls_Enrolment": "Enrolment"})

    # Average
    total = df_subset.groupby("School_Name")[["Total_Attendance", "Total_Enrolment"]].sum().reset_index()
    total["Gender"] = "Average"
    total["Rate"] = (total["Total_Attendance"] / total["Total_Enrolment"].replace(0, pd.NA)) * 100
    total = total.rename(columns={"Total_Attendance": "Attendance", "Total_Enrolment": "Enrolment"})

    # Combine all
    combined = pd.concat([
        boys[["School_Name", "Gender", "Attendance", "Enrolment", "Rate"]],
        girls[["School_Name", "Gender", "Attendance", "Enrolment", "Rate"]],
        total[["School_Name", "Gender", "Attendance", "Enrolment", "Rate"]]
    ])

    combined["Rate"] = combined["Rate"].fillna(0)
    combined["Label"] = combined["Rate"].round(0).astype(int).astype(str) + "%"

    fig2 = px.bar(
        combined,
        x="School_Name",
        y="Rate",
        color="Gender",
        text="Label",
        barmode="group",
        hover_data={
            "School_Name": True,
            "Gender": True,
            "Attendance": True,
            "Enrolment": True
        },
        title=f"Attendance Rate (Boys, Girls & Average) per School — {level}"
    )

    fig2.update_layout(
        xaxis_title="School Name",
        yaxis_title="Attendance Rate (%)",
        xaxis_tickangle=-45,
        height=550
    )
    fig2.show()


C:\Users\JohnKul\AppData\Local\Temp\ipykernel_15408\3136897205.py:122: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

